In [1]:
import gym

from stable_baselines3 import PPO
from stable_baselines3 import SAC
from stable_baselines3 import mSAC
from stable_baselines3.common.evaluation import evaluate_policy, evaluate_meta_policy

import numpy as np
import torch as th
from gym import spaces

##pyfly stuff
from pyfly.pyfly import PyFly
from pyfly.pid_controller import PIDController
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation

In [2]:
th.manual_seed(42)
np.random.seed(666)

In [3]:
import numpy as np
from gym.envs.mujoco import HalfCheetahEnv as HalfCheetahEnv_

class HalfCheetahEnv(HalfCheetahEnv_):
    def _get_obs(self):
        return np.concatenate([
            self.sim.data.qpos.flat[1:],
            self.sim.data.qvel.flat,
            self.get_body_com("torso").flat,
        ]).astype(np.float32).flatten()

    def viewer_setup(self):
        camera_id = self.model.camera_name2id('track')
        self.viewer.cam.type = 2
        self.viewer.cam.fixedcamid = camera_id
        self.viewer.cam.distance = self.model.stat.extent * 0.35
        # Hide the overlay
        self.viewer._hide_overlay = True

    def render(self, mode='human'):
        if mode == 'rgb_array':
            self._get_viewer().render()
            # window size used for old mujoco-py:
            width, height = 500, 500
            data = self._get_viewer().read_pixels(width, height, depth=False)
            return data
        elif mode == 'human':
            self._get_viewer().render()
            
import numpy as np



class HalfCheetahVelEnv(HalfCheetahEnv):
    """Half-cheetah environment with target velocity, as described in [1]. The
    code is adapted from
    https://github.com/cbfinn/maml_rl/blob/9c8e2ebd741cb0c7b8bf2d040c4caeeb8e06cc95/rllab/envs/mujoco/half_cheetah_env_rand.py
    The half-cheetah follows the dynamics from MuJoCo [2], and receives at each
    time step a reward composed of a control cost and a penalty equal to the
    difference between its current velocity and the target velocity. The tasks
    are generated by sampling the target velocities from the uniform
    distribution on [0, 2].
    [1] Chelsea Finn, Pieter Abbeel, Sergey Levine, "Model-Agnostic
        Meta-Learning for Fast Adaptation of Deep Networks", 2017
        (https://arxiv.org/abs/1703.03400)
    [2] Emanuel Todorov, Tom Erez, Yuval Tassa, "MuJoCo: A physics engine for
        model-based control", 2012
        (https://homes.cs.washington.edu/~todorov/papers/TodorovIROS12.pdf)
    """
    def __init__(self, task={}, n_tasks=30000, randomize_tasks=True):
        self._task = task
        self.tasks = self.sample_tasks(n_tasks)
        self._goal_vel = self.tasks[0].get('velocity', 0.0)
        self._goal = self._goal_vel
        self.i = 0
        super(HalfCheetahVelEnv, self).__init__()

    def step(self, action):
        xposbefore = self.sim.data.qpos[0]
        self.do_simulation(action, self.frame_skip)
        xposafter = self.sim.data.qpos[0]

        forward_vel = (xposafter - xposbefore) / self.dt
        forward_reward = -1.0 * abs(forward_vel - self._goal_vel)
        ctrl_cost = 0.5 * 1e-1 * np.sum(np.square(action))

        observation = self._get_obs()
        reward = forward_reward - ctrl_cost
        if self.i >= 200:
            done = True
            self.i = 0
            print(self._goal_vel)
            del self.tasks[0]
            self._goal_vel = self.tasks[0].get('velocity', 0.0)
            self._goal = self._goal_vel
            
        else:
            done = False
            self.i += 1
        infos = dict(reward_forward=forward_reward,
            reward_ctrl=-ctrl_cost, task=self._task)
        return (observation, reward, done, infos)

    def sample_tasks(self, num_tasks):
        np.random.seed(1337)
        print('goal sampled')
        velocities = np.random.choice([1.25, 1.5], num_tasks)#np.random.uniform(0.0, 3.0, size=(num_tasks,))
        tasks = [{'velocity': velocity} for velocity in velocities]
        return tasks

    def get_all_task_idx(self):
        return range(len(self.tasks))

    def reset_task(self, idx):
        self._task = self.tasks[idx]
        self._goal_vel = self._task['velocity']
        self._goal = self._goal_vel
        self.reset()

In [4]:
env = HalfCheetahVelEnv(n_tasks = 5000000)#gym.make('BipedalWalker-v3')#FooEnv()#

meta_model = mSAC('MlpPolicy', env, verbose=1,policy_kwargs=dict(net_arch=[300, 300, 300], latent_dim = 1, hidden_sizes=[200,200,200]))#,learning_rate=0.0006)

for param in meta_model.actor.context_encoder.parameters():
    print(param.data)

meta_reward = []
meta_std = []

print('-Start-')
n_eval =5

meta_model_mean_reward_before, meta_model_std_reward_before = evaluate_meta_policy(meta_model, env, n_eval_episodes=n_eval, add2buff = True)
meta_reward.append(meta_model_mean_reward_before)
meta_std.append(meta_model_std_reward_before)



print('##################################Start Learning##################################')
for i in range(200):
    
    meta_model.learn(total_timesteps=5*200)#, eval_freq=100, n_eval_episodes=5)
    meta_model_mean_reward, meta_model_std_reward = evaluate_meta_policy(meta_model, env, n_eval_episodes=30)

    meta_reward.append(meta_model_mean_reward)
    meta_std.append(meta_model_std_reward)
    
    
    print('epoch:', i)
    print('meta_reward = ', meta_reward)
    print('meta_std = ', meta_std)
    
env.close()

goal sampled


/home/user/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device
Wrapping the env in a DummyVecEnv.
critic with  27
critic with  27
critic with  27
critic with  27
tensor([[ 0.0032, -0.0832, -0.0601,  ..., -0.0273,  0.0409,  0.1588],
        [ 0.0448,  0.0076,  0.0344,  ...,  0.0833, -0.0209, -0.1424],
        [-0.1195, -0.0948, -0.0167,  ...,  0.0039, -0.0861,  0.0050],
        ...,
        [ 0.1500,  0.0135,  0.0602,  ..., -0.0077,  0.0866,  0.1876],
        [-0.0891,  0.1908,  0.1741,  ..., -0.0749,  0.0185, -0.0424],
        [ 0.1467, -0.0597, -0.0749,  ...,  0.1047, -0.1867, -0.0322]])
tensor([-1.8562e-01, -3.2392e-04, -3.1671e-02,  8.3155e-02, -1.1387e-01,
         1.8533e-02, -4.9836e-02, -6.2585e-02,  2.8700e-02,  7.5345e-02,
        -7.2845e-02, -1.3017e-01,  1.3956e-01,  1.2397e-01,  8.1937e-02,
        -1.3313e-01,  6.8562e-02,  1.6363e-01, -2.4181e-02, -3.2732e-02,
         1.1360e-01,  8.0258e-02,  1.1380e-01,  1.1968e-01,  9.8742e-02,
         5.2183e-02, -7.3694e-02, -1.6083e-01,  1.3696e-01, -1.5277e-01,
         1.5

1.5
1.25
1.25
1.5
##################################Start Learning##################################
rollout
collect with prior
collect with prior
1.5
collect with prior
1.5
1.25
1.25
-----------------------------
| time/              |      |
|    episodes        | 4    |
|    fps             | 1051 |
|    time_elapsed    | 0    |
|    total timesteps | 804  |
-----------------------------
apply grads
prior/posterior: 2.3333333333333335
1.25
1.25
1.25
1.5
1.25
1.25
1.25
1.5
1.25
1.5
1.5
1.5
1.25
1.25
1.25
1.25
1.5
1.25
1.5
1.5
1.25
1.5
1.5
1.25
1.5
1.5
1.5
1.5
1.5
1.5
epoch: 0
meta_reward =  [-281.72733962137175, -297.696504188453]
meta_std =  [24.125481031664226, 59.66280792199512]
rollout
collect with prior
collect with prior
1.25
collect with prior
1.25
1.25
1.5
------------------------------------
| time/              |             |
|    episodes        | 4           |
|    fps             | 913         |
|    time_elapsed    | 0           |
|    total timesteps | 804         |
|

apply grads
prior/posterior: 1.1666666666666667
1.5
1.5
1.25
1.25
1.5
1.5
1.5
1.5
1.5
1.5
1.25
1.5
1.25
1.5
1.25
1.25
1.5
1.5
1.25
1.5
1.25
1.25
1.25
1.5
1.5
1.5
1.5
1.25
1.25
1.25
epoch: 7
meta_reward =  [-281.72733962137175, -297.696504188453, -259.55673565166506, -268.864582627905, -265.484702409847, -267.24294241852436, -265.39547059289833, -257.7289832326335, -266.940459063185]
meta_std =  [24.125481031664226, 59.66280792199512, 53.07108299963847, 55.0406295837304, 53.84866968985381, 54.46754096274957, 54.504433577355684, 52.288181159053266, 54.389042122875125]
rollout
collect with prior
collect with prior
1.25
collect with prior
1.5
1.25
1.25
------------------------------------
| time/              |             |
|    episodes        | 4           |
|    fps             | 981         |
|    time_elapsed    | 0           |
|    total timesteps | 804         |
| train/             |             |
|    KL_loss         | 0.018183663 |
|    actor_loss      | 28.4        |
|    avg. 

...

apply grads
prior/posterior: 1.0068027210884354
1.5
1.5
1.25
1.5
1.5
1.25
1.5
1.5
1.5
1.5
1.25
1.25
1.25
1.25
1.5
1.5
1.5
1.5
1.25
1.5
1.25
1.25
1.5
1.5
1.25
1.25
1.5
1.25
1.25
1.5
epoch: 195
meta_reward =  [-281.72733962137175, -297.696504188453, -259.55673565166506, -268.864582627905, -265.484702409847, -267.24294241852436, -265.39547059289833, -257.7289832326335, -266.940459063185, -273.9057017716561, -274.2678100537612, -248.65272555269567, -258.81353352094004, -257.2966821488289, -265.420370833211, -264.50238833299585, -255.32663841422007, -271.26639230516673, -269.6049043430677, -270.6974893169326, -269.4970802909729, -263.17817855923846, -263.4491382029321, -250.9048839199915, -255.32414322387, -259.09662491312315, -258.1390870540691, -244.13361039514317, -268.87536795688646, -253.33670568535683, -249.79840162964143, -258.98901479200913, -265.50113490322224, -280.82552109382397, -244.27294483608335, -258.9527018881678, -265.64536114310454, -282.6273251258732, -257.7144795877255,

apply grads
prior/posterior: 1.0067681895093064
1.25
1.5
1.25
1.25
1.5
1.25
1.5
1.5
1.5
1.5
1.25
1.5
1.5
1.25
1.5
1.25
1.5
1.25
1.25
1.5
1.5
1.25
1.25
1.25
1.5
1.5
1.5
1.5
1.25
1.5
epoch: 196
meta_reward =  [-281.72733962137175, -297.696504188453, -259.55673565166506, -268.864582627905, -265.484702409847, -267.24294241852436, -265.39547059289833, -257.7289832326335, -266.940459063185, -273.9057017716561, -274.2678100537612, -248.65272555269567, -258.81353352094004, -257.2966821488289, -265.420370833211, -264.50238833299585, -255.32663841422007, -271.26639230516673, -269.6049043430677, -270.6974893169326, -269.4970802909729, -263.17817855923846, -263.4491382029321, -250.9048839199915, -255.32414322387, -259.09662491312315, -258.1390870540691, -244.13361039514317, -268.87536795688646, -253.33670568535683, -249.79840162964143, -258.98901479200913, -265.50113490322224, -280.82552109382397, -244.27294483608335, -258.9527018881678, -265.64536114310454, -282.6273251258732, -257.7144795877255,

apply grads
prior/posterior: 1.0067340067340067
1.25
1.25
1.5
1.5
1.5
1.25
1.25
1.5
1.5
1.5
1.5
1.5
1.5
1.5
1.25
1.5
1.5
1.25
1.5
1.5
1.5
1.5
1.25
1.5
1.25
1.25
1.5
1.5
1.25
1.25
epoch: 197
meta_reward =  [-281.72733962137175, -297.696504188453, -259.55673565166506, -268.864582627905, -265.484702409847, -267.24294241852436, -265.39547059289833, -257.7289832326335, -266.940459063185, -273.9057017716561, -274.2678100537612, -248.65272555269567, -258.81353352094004, -257.2966821488289, -265.420370833211, -264.50238833299585, -255.32663841422007, -271.26639230516673, -269.6049043430677, -270.6974893169326, -269.4970802909729, -263.17817855923846, -263.4491382029321, -250.9048839199915, -255.32414322387, -259.09662491312315, -258.1390870540691, -244.13361039514317, -268.87536795688646, -253.33670568535683, -249.79840162964143, -258.98901479200913, -265.50113490322224, -280.82552109382397, -244.27294483608335, -258.9527018881678, -265.64536114310454, -282.6273251258732, -257.7144795877255, -

apply grads
prior/posterior: 1.0067001675041876
1.5
1.25
1.5
1.25
1.5
1.25
1.5
1.5
1.25
1.5
1.25
1.5
1.25
1.5
1.25
1.25
1.25
1.25
1.25
1.5
1.5
1.5
1.25
1.5
1.5
1.5
1.25
1.25
1.25
1.5
epoch: 198
meta_reward =  [-281.72733962137175, -297.696504188453, -259.55673565166506, -268.864582627905, -265.484702409847, -267.24294241852436, -265.39547059289833, -257.7289832326335, -266.940459063185, -273.9057017716561, -274.2678100537612, -248.65272555269567, -258.81353352094004, -257.2966821488289, -265.420370833211, -264.50238833299585, -255.32663841422007, -271.26639230516673, -269.6049043430677, -270.6974893169326, -269.4970802909729, -263.17817855923846, -263.4491382029321, -250.9048839199915, -255.32414322387, -259.09662491312315, -258.1390870540691, -244.13361039514317, -268.87536795688646, -253.33670568535683, -249.79840162964143, -258.98901479200913, -265.50113490322224, -280.82552109382397, -244.27294483608335, -258.9527018881678, -265.64536114310454, -282.6273251258732, -257.714479587725

1.5
------------------------------------
| time/              |             |
|    episodes        | 4           |
|    fps             | 982         |
|    time_elapsed    | 0           |
|    total timesteps | 804         |
| train/             |             |
|    KL_loss         | 6.4738245   |
|    actor_loss      | 467         |
|    avg. z          | -1.2029169  |
|    avg. z var      | 0.011806675 |
|    critic_loss     | 666         |
|    ent_coef        | 0.0923      |
|    ent_coef_loss   | -0.585      |
|    learning_rate   | 0.0003      |
|    n_updates       | 39800       |
------------------------------------
apply grads
prior/posterior: 1.0066666666666666
1.25
1.5
1.5
1.5
1.5
1.5
1.25
1.25
1.25
1.25
1.5
1.25
1.25
1.5
1.25
1.25
1.5
1.25
1.5
1.5
1.5
1.5
1.25
1.25
1.25
1.5
1.25
1.5
1.25
1.5
epoch: 199
meta_reward =  [-281.72733962137175, -297.696504188453, -259.55673565166506, -268.864582627905, -265.484702409847, -267.24294241852436, -265.39547059289833, -257.72898323263

In [5]:
# since last time the sampling in evaluation and the sampling depth für context in train

In [6]:
a = [0.7860740250467451, 1.3779506616437, 1.5551784617926114, 0.7858287766954339, 2.928255854632301, 2.1984436580714473, 0.3458226800494484, 1.1588252059030784, 1.885503538619135, 1.3296746059353843, 2.3686750260717098, 2.38235572737476, 2.1685728538042, 2.711781597991056, 1.999393957709451, 1.8307119965196041, 1.260624986955272, 0.2671352016074815, 1.402548298157229, 2.4971121092039477, 2.279859532934837, 2.839511756498454, 0.37517377900679827, 0.5634252162792469, 1.1076871867784575, 0.3762089882646701, 0.9730737106816765, 1.59509821076111, 2.73336720468094, 1.7527743843309642, 1.248311818712994, 1.4736872317976357, 1.3468935565348823, 1.1823408005585798, 2.107571159141275, 2.731734949135858, 0.37990433907251275, 0.7641724093376423, 1.512021117484312, 1.0545722993955051, 1.6782903704242598, 2.8169478016378617, 1.2892761911000037, 1.4989447841160373, 2.992167429662661, 0.47605191646339684, 0.8343795584830792, 0.9630016215605014, 0.5557528352161346, 0.008703090506129718, 0.48863259980930185, 0.5975857056152952, 0.7963510149605818, 0.864501443436107, 0.047374356635443315, 1.2762439954335498, 2.010656574480702, 2.9203781981030765, 2.100895260148338, 2.3680082550723363, 1.6935569384236153, 2.1718436939592927, 2.728960574484791, 0.7084476270060592, 1.480194734343588, 1.022807914959746, 1.9006458178251902, 1.532128187898675, 1.598488390876416, 1.403537916973332, 2.979915119672488, 2.2720545866630646, 0.5357060314905813, 1.474444589494259, 0.48635227727992836, 1.5568759511574917, 2.2805153214175724, 2.0142083586378923, 2.3757763016921394, 2.8720776710137823, 2.195070206002873, 2.079658332603392, 2.9186169896064182, 2.1386625004738615, 0.17603797674033772, 0.5324039546451034, 1.124502553740213, 1.0493076380999455, 2.379069097887579, 0.5713365770889663, 2.9506458154309234, 2.6917428928705496, 0.7125322952456615, 1.0837847172522923, 1.2394239042739765, 1.531412840477841, 1.188193847652982, 2.5079036400976444, 0.043133571081973576, 0.03269028711044775]

In [7]:
len(a)

100

In [8]:
a =[0.7860740250467451, 1.3779506616437, 1.5551784617926114, 0.7858287766954339, 2.928255854632301, 2.1984436580714473, 0.3458226800494484, 1.1588252059030784, 1.885503538619135, 1.3296746059353843, 2.3686750260717098, 2.38235572737476, 2.1685728538042, 2.711781597991056, 2.9784953809599912, 2.096870367728471, 2.04652104928077, 1.999393957709451, 1.8307119965196041, 1.4436558931536259, 1.260624986955272, 1.8408117163039968, 1.0474247818307383, 0.2893499571670589, 0.2671352016074815, 1.402548298157229, 2.4971121092039477, 2.279859532934837, 2.839511756498454, 0.37517377900679827, 0.5634252162792469, 1.1076871867784575, 2.7450762138519322, 2.176699864999294, 0.3762089882646701, 0.9730737106816765, 1.59509821076111, 2.73336720468094, 2.050959316523306, 2.809525172141024, 1.7527743843309642, 1.248311818712994, 1.4736872317976357, 1.3468935565348823, 1.1823408005585798, 2.107571159141275, 2.731734949135858, 0.37990433907251275, 0.7641724093376423, 1.512021117484312, 1.0545722993955051, 1.6782903704242598, 2.8169478016378617, 1.2892761911000037, 0.8861106117834646, 1.4989447841160373, 2.992167429662661, 0.7015287662209706, 0.11168961358780638, 0.47605191646339684, 0.8343795584830792, 0.9630016215605014, 0.5557528352161346, 0.008703090506129718, 0.48863259980930185, 0.5975857056152952, 0.7963510149605818, 0.864501443436107, 0.047374356635443315, 1.2762439954335498, 2.010656574480702, 2.9203781981030765, 2.100895260148338, 2.3680082550723363, 1.6935569384236153, 2.1718436939592927, 2.728960574484791, 0.7084476270060592, 1.480194734343588, 0.09577530004016699, 1.022807914959746, 1.9006458178251902, 1.532128187898675, 1.598488390876416, 1.454276543853031, 1.403537916973332, 1.071058434727445, 2.979915119672488, 2.2720545866630646, 0.5357060314905813, 1.474444589494259, 0.48635227727992836, 1.5568759511574917, 2.2805153214175724, 2.0142083586378923, 2.3757763016921394, 2.2093259883723895, 0.7794378763219836, 2.8720776710137823, 2.195070206002873, 2.079658332603392, 2.9186169896064182, 2.1386625004738615, 2.802629411145565, 0.17603797674033772, 0.5324039546451034, 1.124502553740213, 1.0493076380999455, 1.744195890973208, 0.5583223214918457, 2.379069097887579, 0.08306914545160993, 0.5713365770889663, 1.2224346928609924, 1.1705025304184877, 0.14470529568442736, 2.9506458154309234, 2.6917428928705496, 2.5617488625541105, 2.317384380319182, 0.7125322952456615, 0.3765232265537465, 1.0837847172522923, 1.2394239042739765, 1.531412840477841, 1.188193847652982, 2.5079036400976444, 0.043133571081973576, 0.03269028711044775, 0.7144484446282852]

In [9]:
int(6029/201)

29